In [0]:
import pandas as pd

#1. Revenue per Recognition

In [0]:
revenue = spark.table("silver.revenue.revenue")
apartments = spark.table("silver.backoffice.apartments")
contracts = spark.table("silver.backoffice.contracts")
bookings = spark.table("silver.backoffice.bookings")

revenue = revenue.toPandas()
apartments = apartments.toPandas()
contracts = contracts.toPandas()
bookings = bookings.toPandas()

##1.1. Rent Revenue

###1.1.1. Actuals
###### (Rent Revenue)

In [0]:
rent_per_recognition = """
WITH NON_PRORRATED AS (
        SELECT 
            (COALESCE(SUM(rent),0)) rent, trunc(revenue.day, 'month') month, COALESCE(revenue.contract_id, NULL) contract_id, city, contracts.inserted_at as sell_date, contracts.booking_id, apartments.codename, bookings.state
        
        FROM silver.revenue.revenue
            LEFT JOIN silver.backoffice.apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings ON contracts.booking_id = bookings.id
            
        WHERE bookings.state NOT IN ('canceled', 'denied')
        
        GROUP BY trunc(revenue.day, 'month'), city, contracts.inserted_at, contracts.booking_id, apartments.codename, bookings.state, COALESCE(revenue.contract_id, NULL)
    )

    ,New_R AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(rent)/1.1 
                WHEN city IN ('Berlin') THEN SUM(rent)/1.07 
                ELSE SUM(rent) END AS rent, city, month, sell_date, booking_id, codename, state, contract_id
        FROM NON_PRORRATED 
        GROUP BY city, month, sell_date, booking_id, codename, state, contract_id
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )

    SELECT city, month, sell_date, booking_id, codename, contract_id, rent FROM New_R WHERE month >= DATE('2024-12-01')
    UNION ALL
    SELECT city, month, sell_date, booking_id, codename, contract_id, rent FROM silver.finance.materialized_rent_per_recognition
"""

spark.sql(rent_per_recognition).write.mode("overwrite").saveAsTable("silver.finance.rent_per_recognition")

#TEST_EDGAR_1:
    #####################################################################
    #rent = revenue[['rent', 'day', 'contract_id', 'apartment_id']]
    #apartments = apartments[['id', 'city', 'codename']]
    #contracts = contracts[['id', 'booking_id', 'inserted_at']]
    #bookings = bookings[(bookings['state'] != 'canceled') & (bookings['state'] != 'denied')][['id', 'state']]
    #
    #
    #####################################################################
    #NON_PRORRATED_R = pd.merge(rent, apartments, how='left', left_on='apartment_id', right_on='id')
    #NON_PRORRATED_R = pd.merge(NON_PRORRATED_R, contracts, how='left', left_on='contract_id', right_on='id')
    #NON_PRORRATED_R = pd.merge(bookings, NON_PRORRATED_R, how='left', left_on='id', right_on='booking_id').drop#(['id_x', 'id_y', 'state'], axis=1)
    #
    #
    #####################################################################
    #NON_PRORRATED_R['month'] = pd.to_datetime(NON_PRORRATED_R['day'],format='%Y-%m-%d').apply(lambda x: x.replace#(day=1))
    #NON_PRORRATED_R['day'] = pd.to_datetime(NON_PRORRATED_R['day'],format='%Y-%m-%d')
    #NON_PRORRATED_R['rent'] = NON_PRORRATED_R['rent'].fillna(0)
    #
    #
    #####################################################################
    #NON_PRORRATED_R = (
    #    NON_PRORRATED_R.groupby(['city', 'month', 'inserted_at', 'booking_id', 'codename', 'contract_id'])
    #    .agg(rent=('rent', 'sum'))
    #    .reset_index() 
    #)
    #
    #
    #####################################################################
    #def vat(NON_PRORRATED_R):
    #    if NON_PRORRATED_R['city'] in ['Barcelona', 'Madrid']:
    #        return NON_PRORRATED_R['rent'] / 1.1
    #    elif NON_PRORRATED_R['city'] in ['Belin']:
    #        return NON_PRORRATED_R['rent'] / 1.07
    #    else:
    #        return NON_PRORRATED_R['rent']
    #    
    #NON_PRORRATED_R['rent'] = NON_PRORRATED_R.apply(vat, axis=1)
    #
    #New_R = (
    #    NON_PRORRATED_R.groupby(['city', 'month', 'inserted_at', 'booking_id', 'codename', 'contract_id'])
    #    .agg(rent=('rent', 'sum'))
    #    .reset_index()
    #)
    #
    #
    #####################################################################
    #New_R = New_R.rename(columns={'inserted_at': 'sell_date'})
    #
    #New_R = spark.createDataFrame(New_R)
    #
    #New_R.write.mode("overwrite").saveAsTable("silver.finance.rent_per_recognition_test")

###1.1.2. Deltas 7 Days Ago
###### (Rent Revenue)

In [0]:
rent_per_recognition_7_days_deltas = """
WITH NON_PRORRATED AS (
        SELECT 
            (COALESCE(SUM(rent),0)) rent, trunc(revenue.day, 'month') month, COALESCE(revenue.contract_id, NULL) contract_id, city, contracts.inserted_at as sell_date, contracts.booking_id, apartments.codename, bookings.state
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE bookings.state NOT IN ('canceled', 'denied')
        
        GROUP BY trunc(revenue.day, 'month'), city, contracts.inserted_at, contracts.booking_id, apartments.codename, bookings.state, COALESCE(revenue.contract_id, NULL)
    )

    ,New_R AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(rent)/1.1 
                WHEN city IN ('Berlin') THEN SUM(rent)/1.07 
                ELSE SUM(rent) END AS rent, city, month, sell_date, booking_id, codename, state, contract_id
        FROM NON_PRORRATED 
        GROUP BY city, month, sell_date, booking_id, codename, state, contract_id
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )

    SELECT city, month, sell_date, booking_id, codename, contract_id, rent FROM New_R
"""

spark.sql(rent_per_recognition_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.rent_per_recognition_7_days_deltas")


###1.1.3. Deltas Since Begining of Month
###### (Rent Revenue)


In [0]:
rent_per_recognition_begining_month_deltas = """
WITH NON_PRORRATED AS (
        SELECT 
            (COALESCE(SUM(rent),0)) rent, trunc(revenue.day, 'month') month, COALESCE(revenue.contract_id, NULL) contract_id, city, contracts.inserted_at as sell_date, contracts.booking_id, apartments.codename, bookings.state
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE bookings.state NOT IN ('canceled', 'denied')
        
        GROUP BY trunc(revenue.day, 'month'), city, contracts.inserted_at, contracts.booking_id, apartments.codename, bookings.state, COALESCE(revenue.contract_id, NULL)
    )

    ,New_R AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(rent)/1.1 
                WHEN city IN ('Berlin') THEN SUM(rent)/1.07 
                ELSE SUM(rent) END AS rent, city, month, sell_date, booking_id, codename, state, contract_id
        FROM NON_PRORRATED 
        GROUP BY city, month, sell_date, booking_id, codename, state, contract_id
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )

    SELECT city, month, sell_date, booking_id, codename, contract_id, rent FROM New_R
"""

spark.sql(rent_per_recognition_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.rent_per_recognition_begining_month_deltas")

##1.2. Utilities Revenue

###1.2.1. Actuals
###### (Utilities Revenue)

In [0]:
utilities_per_recognition = """
WITH NON_PRORRATED AS (
        SELECT 
            COALESCE(SUM(utilities),0) utilities, trunc(revenue.day, 'month') month, city, contracts.inserted_at as sell_date, contracts.booking_id, apartments.codename,
            COALESCE(revenue.contract_id, NULL) contract_id
        
        FROM silver.revenue.revenue
            LEFT JOIN silver.backoffice.apartments ON revenue.revenue.apartment_id = backoffice.apartments.id
            LEFT JOIN silver.backoffice.contracts ON revenue.revenue.contract_id = backoffice.contracts.id
            LEFT JOIN silver.backoffice.bookings ON backoffice.contracts.booking_id = backoffice.bookings.id
            
        WHERE backoffice.bookings.state NOT IN ('canceled', 'denied')
            
        GROUP BY trunc(revenue.day, 'month'), city, contracts.inserted_at, contracts.booking_id, apartments.codename, revenue.contract_id
    )

    ,New AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(utilities)/1.1 
                WHEN city IN ('Berlin') THEN SUM(utilities)/1.07 
                ELSE SUM(utilities) END AS utilities, city, month, sell_date, booking_id, codename, contract_id  
        FROM NON_PRORRATED 
        --WHERE month >= trunc(current_date, 'month') - INTERVAL 12 MONTH AND month <= trunc(current_date, 'month') + INTERVAL 4 MONTH
        GROUP BY city, month, sell_date, booking_id, codename, contract_id  
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )

    SELECT city, month, sell_date, booking_id, codename, contract_id, utilities FROM New WHERE month >= DATE('2024-12-01')
    UNION ALL
    SELECT city, month, sell_date, booking_id, codename, contract_id, utilities FROM silver.finance.materialized_utilities_per_recognition
"""

spark.sql(utilities_per_recognition).write.mode("overwrite").saveAsTable("silver.finance.utilities_per_recognition")

#TEST_EDGAR_2:
    #####################################################################
    #utilities = revenue[['utilities', 'day', 'contract_id', 'apartment_id']]

    #####################################################################
    #NON_PRORRATED_U = pd.merge(utilities, apartments, how='left', left_on='apartment_id', right_on='id')
    #NON_PRORRATED_U = pd.merge(NON_PRORRATED_U, contracts, how='left', left_on='contract_id', right_on='id')
    #NON_PRORRATED_U = pd.merge(bookings, NON_PRORRATED_U, how='left', left_on='id', right_on='booking_id').#drop(['id_x', 'id_y', 'state'], axis=1)

    #####################################################################
    #NON_PRORRATED_U['month'] = pd.to_datetime(NON_PRORRATED_U['day'],format='%Y-%m-%d').apply(lambda x: x.#replace(day=1))
    #NON_PRORRATED_U['day'] = pd.to_datetime(NON_PRORRATED_U['day'],format='%Y-%m-%d')
    #NON_PRORRATED_U['utilities'] = NON_PRORRATED_U['utilities'].fillna(0)

    #####################################################################
    #NON_PRORRATED_U = (
    #    NON_PRORRATED_U.groupby(['city', 'month', 'inserted_at', 'booking_id', 'codename', 'contract_id'])
    #    .agg(utilities=('utilities', 'sum'))
    #    .reset_index() 
    #)

    #####################################################################
    #def vat(NON_PRORRATED_U):
    #    if NON_PRORRATED_U['city'] in ['Barcelona', 'Madrid']:
    #        return NON_PRORRATED_U['utilities'] / 1.1
    #    elif NON_PRORRATED_U['city'] in ['Belin']:
    #        return NON_PRORRATED_U['utilities'] / 1.07
    #    else:
    #        return NON_PRORRATED_U['utilities']
    #    
    #NON_PRORRATED_U['utilities'] = NON_PRORRATED_U.apply(vat, axis=1)

    #New_U = (
    #    NON_PRORRATED_U.groupby(['city', 'month', 'inserted_at', 'booking_id', 'codename', 'contract_id'])
    #    .agg(utilities=('utilities', 'sum'))
    #    .reset_index()
    #)

    #####################################################################
    #New_U = New_U.rename(columns={'inserted_at': 'sell_date'})

    #New_U = spark.createDataFrame(New_U)

    #New_U.write.mode("overwrite").saveAsTable("silver.finance.utilities_per_recognition_test")

###1.2.2. Deltas 7 Days Ago
###### (Utilities Revenue)

In [0]:
utilities_per_recognition_7_days_deltas = """
WITH NON_PRORRATED AS (
        SELECT 
            COALESCE(SUM(utilities),0) utilities, trunc(revenue.day, 'month') month, city, contracts.inserted_at as sell_date, contracts.booking_id, apartments.codename,
            COALESCE(revenue.contract_id, NULL) contract_id
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE bookings.state NOT IN ('canceled', 'denied')
            
        GROUP BY trunc(revenue.day, 'month'), city, contracts.inserted_at, contracts.booking_id, apartments.codename, revenue.contract_id
    )

    ,New AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(utilities)/1.1 
                WHEN city IN ('Berlin') THEN SUM(utilities)/1.07 
                ELSE SUM(utilities) END AS utilities, city, month, sell_date, booking_id, codename, contract_id  
        FROM NON_PRORRATED 
        --WHERE month >= trunc(current_date, 'month') - INTERVAL 12 MONTH AND month <= trunc(current_date, 'month') + INTERVAL 4 MONTH
        GROUP BY city, month, sell_date, booking_id, codename, contract_id  
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )

    SELECT city, month, sell_date, booking_id, codename, contract_id, utilities FROM New
"""

spark.sql(utilities_per_recognition_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.utilities_per_recognition_7_days_deltas")

###1.2.3. Deltas Since Begining of Month
###### (Utilities Revenue)

In [0]:
utilities_per_recognition_begining_month_deltas = """
WITH NON_PRORRATED AS (
        SELECT 
            COALESCE(SUM(utilities),0) utilities, trunc(revenue.day, 'month') month, city, contracts.inserted_at as sell_date, contracts.booking_id, apartments.codename,
            COALESCE(revenue.contract_id, NULL) contract_id
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE bookings.state NOT IN ('canceled', 'denied')
            
        GROUP BY trunc(revenue.day, 'month'), city, contracts.inserted_at, contracts.booking_id, apartments.codename, revenue.contract_id
    )

    ,New AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(utilities)/1.1 
                WHEN city IN ('Berlin') THEN SUM(utilities)/1.07 
                ELSE SUM(utilities) END AS utilities, city, month, sell_date, booking_id, codename, contract_id  
        FROM NON_PRORRATED 
        --WHERE month >= trunc(current_date, 'month') - INTERVAL 12 MONTH AND month <= trunc(current_date, 'month') + INTERVAL 4 MONTH
        GROUP BY city, month, sell_date, booking_id, codename, contract_id  
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )

    SELECT city, month, sell_date, booking_id, codename, contract_id, utilities FROM New
"""

spark.sql(utilities_per_recognition_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.utilities_per_recognition_begining_month_deltas")

##1.3. Service Charge Revenue

###1.3.1. Actuals
###### (Service Charge Revenue)

In [0]:
sc_per_recognition = """
WITH NON_PRORRATED AS (
        SELECT 
            COALESCE(SUM(service_charge),0) service_charge, trunc(contracts.inserted_at, 'month') sell_date_2, city, 
            contracts.inserted_at as sell_date, contracts.booking_id, apartments.codename,
            COALESCE(revenue.contract_id, NULL) contract_id
        
        FROM silver.revenue.revenue
            LEFT JOIN silver.backoffice.apartments ON revenue.revenue.apartment_id = backoffice.apartments.id
            LEFT JOIN silver.backoffice.contracts ON revenue.revenue.contract_id = backoffice.contracts.id
            LEFT JOIN silver.backoffice.bookings ON backoffice.contracts.booking_id = backoffice.bookings.id
            
        WHERE city IN ('Barcelona', 'Madrid', 'Lisbon', 'Berlin')
            AND bookings.source NOT IN('Apartool', 'Silverdoor', 'Nomad', 'RelocatEU', 'Dwellworks', 'Altovita', 'Airbnb', 'HomeLike', 'VRBO')
            AND backoffice.bookings.state NOT IN ('canceled', 'denied')
        
        GROUP BY trunc(contracts.inserted_at, 'month'), city, contracts.inserted_at, contracts.booking_id, apartments.codename, COALESCE(revenue.contract_id, NULL)
    )

    ,PRORRATED_5 AS (
        SELECT 
            COALESCE(SUM(service_charge),0) service_charge, city, contracts.start_date, contracts.end_date, 
            revenue.contract_id, contracts.inserted_at sell_date, contracts.booking_id, apartments.codename
        
        FROM silver.revenue.revenue
            LEFT JOIN silver.backoffice.apartments ON revenue.revenue.apartment_id = backoffice.apartments.id
            LEFT JOIN silver.backoffice.contracts ON revenue.revenue.contract_id = backoffice.contracts.id
            LEFT JOIN silver.backoffice.bookings ON backoffice.contracts.booking_id = backoffice.bookings.id
            
        WHERE (city NOT IN ('Barcelona', 'Madrid', 'Lisbon', 'Berlin') OR bookings.source IN ('Apartool', 'Silverdoor', 'Nomad', 'RelocatEU', 'Dwellworks', 'Altovita', 'Airbnb', 'HomeLike', 'VRBO'))
            AND backoffice.bookings.state NOT IN ('canceled', 'denied')
        
        GROUP BY city, contracts.start_date, contracts.end_date, revenue.contract_id, contracts.inserted_at, contracts.booking_id, apartments.codename
    )

    ,PRORRATED_4 AS (
        SELECT 
            explode(
                sequence(
                    trunc(start_date, 'month'),
                    trunc(end_date, 'month'),
                    INTERVAL 1 MONTH
                )
            ) month, city, service_charge::DECIMAL / (end_date::DATE - start_date::DATE)::INTEGER AS service_charge, start_date check_in, end_date check_out, contract_id,
            sell_date, booking_id, codename
            
        FROM PRORRATED_5
    )

    ,PRORRATED_3 AS (
        SELECT
            *, 
            CASE
                WHEN (month) = trunc(check_out::date, 'month')::date and trunc(check_in::date, 'month') <> trunc(check_out::date, 'month') THEN EXTRACT(DAY FROM check_out::date -  trunc((month)::DATE, 'MONTH')) ::INTEGER
                WHEN (month) = trunc(check_in::date, 'month')::date and trunc(check_in::date, 'month') <> trunc(check_out::date, 'month') THEN EXTRACT(DAY FROM trunc((month + INTERVAL '1 month')::DATE, 'MONTH') - check_in)::INTEGER
                WHEN trunc(check_out::date, 'month') = trunc(check_in::date, 'month') THEN (check_out::DATE - check_in::DATE)::INTEGER
                ELSE EXTRACT(DAY FROM trunc((month + INTERVAL '1 month')::DATE, 'MONTH') - month)::INTEGER
            END as nights
            
        FROM PRORRATED_4
    )

    ,PRORRATED_2 as (
        SELECT 
            city, service_charge::DECIMAL * nights service_charge, month, sell_date, booking_id, codename, contract_id
        
        FROM PRORRATED_3
    )

    ,PRORRATED AS (
        SELECT SUM(service_charge) service_charge, city, month, sell_date, booking_id, codename, contract_id FROM PRORRATED_2 GROUP BY city, month, sell_date, contract_id, booking_id, codename
    )

    ,JOINED AS (
        SELECT city, service_charge, month, sell_date, booking_id, codename, contract_id FROM PRORRATED
        UNION ALL 
        SELECT city, service_charge, sell_date_2 month, sell_date, booking_id, codename, contract_id FROM NON_PRORRATED
    )

    ,New AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(service_charge)/1.1 
                WHEN city IN ('Berlin') THEN SUM(service_charge)/1.07 
                ELSE SUM(service_charge) END AS service_charge, city, month, sell_date, booking_id, codename, contract_id 
        FROM JOINED 
        --WHERE month >= trunc(current_date, 'month') - INTERVAL 12 MONTH AND month <= trunc(current_date, 'month') + INTERVAL 4 MONTH
        GROUP BY city, month, sell_date, booking_id, codename, contract_id 
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )

    SELECT city, month, sell_date, booking_id, codename, contract_id, service_charge FROM New WHERE month >= DATE('2024-12-01')
    UNION ALL
    SELECT city, month, sell_date, booking_id, codename, contract_id, service_charge FROM silver.finance.materialized_sc_per_recognition
"""

spark.sql(sc_per_recognition).write.mode("overwrite").saveAsTable("silver.finance.sc_per_recognition")

###1.3.2. Deltas 7 Days Ago
###### (Service Charge Revenue)

In [0]:
sc_per_recognition_7_days_deltas = """
WITH NON_PRORRATED AS (
        SELECT 
            COALESCE(SUM(service_charge),0) service_charge, trunc(contracts.inserted_at, 'month') sell_date_2, city, 
            contracts.inserted_at as sell_date, contracts.booking_id, apartments.codename,
            COALESCE(revenue.contract_id, NULL) contract_id
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE city IN ('Barcelona', 'Madrid', 'Lisbon', 'Berlin')
            AND bookings.source NOT IN('Apartool', 'Silverdoor', 'Nomad', 'RelocatEU', 'Dwellworks', 'Altovita', 'Airbnb', 'HomeLike', 'VRBO')
            AND bookings.state NOT IN ('canceled', 'denied')
        
        GROUP BY trunc(contracts.inserted_at, 'month'), city, contracts.inserted_at, contracts.booking_id, apartments.codename, COALESCE(revenue.contract_id, NULL)
    )

    ,PRORRATED_5 AS (
        SELECT 
            COALESCE(SUM(service_charge),0) service_charge, city, contracts.start_date, contracts.end_date, 
            revenue.contract_id, contracts.inserted_at sell_date, contracts.booking_id, apartments.codename
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE (city NOT IN ('Barcelona', 'Madrid', 'Lisbon', 'Berlin') OR bookings.source IN ('Apartool', 'Silverdoor', 'Nomad', 'RelocatEU', 'Dwellworks', 'Altovita', 'Airbnb', 'HomeLike', 'VRBO'))
            AND bookings.state NOT IN ('canceled', 'denied')
        
        GROUP BY city, contracts.start_date, contracts.end_date, revenue.contract_id, contracts.inserted_at, contracts.booking_id, apartments.codename
    )

    ,PRORRATED_4 AS (
        SELECT 
            explode(
                sequence(
                    trunc(start_date, 'month'),
                    trunc(end_date, 'month'),
                    INTERVAL 1 MONTH
                )
            ) month, city, service_charge::DECIMAL / (end_date::DATE - start_date::DATE)::INTEGER AS service_charge, start_date check_in, end_date check_out, contract_id,
            sell_date, booking_id, codename
            
        FROM PRORRATED_5
    )

    ,PRORRATED_3 AS (
        SELECT
            *, 
            CASE
                WHEN (month) = trunc(check_out::date, 'month')::date and trunc(check_in::date, 'month') <> trunc(check_out::date, 'month') THEN EXTRACT(DAY FROM check_out::date -  trunc((month)::DATE, 'MONTH')) ::INTEGER
                WHEN (month) = trunc(check_in::date, 'month')::date and trunc(check_in::date, 'month') <> trunc(check_out::date, 'month') THEN EXTRACT(DAY FROM trunc((month + INTERVAL '1 month')::DATE, 'MONTH') - check_in)::INTEGER
                WHEN trunc(check_out::date, 'month') = trunc(check_in::date, 'month') THEN (check_out::DATE - check_in::DATE)::INTEGER
                ELSE EXTRACT(DAY FROM trunc((month + INTERVAL '1 month')::DATE, 'MONTH') - month)::INTEGER
            END as nights
            
        FROM PRORRATED_4
    )

    ,PRORRATED_2 as (
        SELECT 
            city, service_charge::DECIMAL * nights service_charge, month, sell_date, booking_id, codename, contract_id
        
        FROM PRORRATED_3
    )

    ,PRORRATED AS (
        SELECT SUM(service_charge) service_charge, city, month, sell_date, booking_id, codename, contract_id FROM PRORRATED_2 GROUP BY city, month, sell_date, contract_id, booking_id, codename
    )

    ,JOINED AS (
        SELECT city, service_charge, month, sell_date, booking_id, codename, contract_id FROM PRORRATED
        UNION ALL 
        SELECT city, service_charge, sell_date_2 month, sell_date, booking_id, codename, contract_id FROM NON_PRORRATED
    )

    ,New AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(service_charge)/1.1 
                WHEN city IN ('Berlin') THEN SUM(service_charge)/1.07 
                ELSE SUM(service_charge) END AS service_charge, city, month, sell_date, booking_id, codename, contract_id 
        FROM JOINED 
        --WHERE month >= trunc(current_date, 'month') - INTERVAL 12 MONTH AND month <= trunc(current_date, 'month') + INTERVAL 4 MONTH
        GROUP BY city, month, sell_date, booking_id, codename, contract_id 
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )

    SELECT city, month, sell_date, booking_id, codename, contract_id, service_charge FROM New
"""

spark.sql(sc_per_recognition_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.sc_per_recognition_7_days_deltas")

###1.3.3. Deltas Since Begining of Month
###### (Service Charge Revenue)

In [0]:
sc_per_recognition_begining_month_deltas = """
WITH NON_PRORRATED AS (
        SELECT 
            COALESCE(SUM(service_charge),0) service_charge, trunc(contracts.inserted_at, 'month') sell_date_2, city, 
            contracts.inserted_at as sell_date, contracts.booking_id, apartments.codename,
            COALESCE(revenue.contract_id, NULL) contract_id
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE city IN ('Barcelona', 'Madrid', 'Lisbon', 'Berlin')
            AND bookings.source NOT IN('Apartool', 'Silverdoor', 'Nomad', 'RelocatEU', 'Dwellworks', 'Altovita', 'Airbnb', 'HomeLike', 'VRBO')
            AND bookings.state NOT IN ('canceled', 'denied')
        
        GROUP BY trunc(contracts.inserted_at, 'month'), city, contracts.inserted_at, contracts.booking_id, apartments.codename, COALESCE(revenue.contract_id, NULL)
    )

    ,PRORRATED_5 AS (
        SELECT 
            COALESCE(SUM(service_charge),0) service_charge, city, contracts.start_date, contracts.end_date, 
            revenue.contract_id, contracts.inserted_at sell_date, contracts.booking_id, apartments.codename
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE (city NOT IN ('Barcelona', 'Madrid', 'Lisbon', 'Berlin') OR bookings.source IN ('Apartool', 'Silverdoor', 'Nomad', 'RelocatEU', 'Dwellworks', 'Altovita', 'Airbnb', 'HomeLike', 'VRBO'))
            AND bookings.state NOT IN ('canceled', 'denied')
        
        GROUP BY city, contracts.start_date, contracts.end_date, revenue.contract_id, contracts.inserted_at, contracts.booking_id, apartments.codename
    )

    ,PRORRATED_4 AS (
        SELECT 
            explode(
                sequence(
                    trunc(start_date, 'month'),
                    trunc(end_date, 'month'),
                    INTERVAL 1 MONTH
                )
            ) month, city, service_charge::DECIMAL / (end_date::DATE - start_date::DATE)::INTEGER AS service_charge, start_date check_in, end_date check_out, contract_id,
            sell_date, booking_id, codename
            
        FROM PRORRATED_5
    )

    ,PRORRATED_3 AS (
        SELECT
            *, 
            CASE
                WHEN (month) = trunc(check_out::date, 'month')::date and trunc(check_in::date, 'month') <> trunc(check_out::date, 'month') THEN EXTRACT(DAY FROM check_out::date -  trunc((month)::DATE, 'MONTH')) ::INTEGER
                WHEN (month) = trunc(check_in::date, 'month')::date and trunc(check_in::date, 'month') <> trunc(check_out::date, 'month') THEN EXTRACT(DAY FROM trunc((month + INTERVAL '1 month')::DATE, 'MONTH') - check_in)::INTEGER
                WHEN trunc(check_out::date, 'month') = trunc(check_in::date, 'month') THEN (check_out::DATE - check_in::DATE)::INTEGER
                ELSE EXTRACT(DAY FROM trunc((month + INTERVAL '1 month')::DATE, 'MONTH') - month)::INTEGER
            END as nights
            
        FROM PRORRATED_4
    )

    ,PRORRATED_2 as (
        SELECT 
            city, service_charge::DECIMAL * nights service_charge, month, sell_date, booking_id, codename, contract_id
        
        FROM PRORRATED_3
    )

    ,PRORRATED AS (
        SELECT SUM(service_charge) service_charge, city, month, sell_date, booking_id, codename, contract_id FROM PRORRATED_2 GROUP BY city, month, sell_date, contract_id, booking_id, codename
    )

    ,JOINED AS (
        SELECT city, service_charge, month, sell_date, booking_id, codename, contract_id FROM PRORRATED
        UNION ALL 
        SELECT city, service_charge, sell_date_2 month, sell_date, booking_id, codename, contract_id FROM NON_PRORRATED
    )

    ,New AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(service_charge)/1.1 
                WHEN city IN ('Berlin') THEN SUM(service_charge)/1.07 
                ELSE SUM(service_charge) END AS service_charge, city, month, sell_date, booking_id, codename, contract_id 
        FROM JOINED 
        --WHERE month >= trunc(current_date, 'month') - INTERVAL 12 MONTH AND month <= trunc(current_date, 'month') + INTERVAL 4 MONTH
        GROUP BY city, month, sell_date, booking_id, codename, contract_id 
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )

    SELECT city, month, sell_date, booking_id, codename, contract_id, service_charge FROM New
"""

spark.sql(sc_per_recognition_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.sc_per_recognition_begining_month_deltas")

##1.4. Penalty Revenue

###1.4.1. Actuals
###### (Penalty Revenue)

In [0]:
penalty_per_recognition = """
WITH NON_PRORRATED AS (
        SELECT 
            COALESCE(SUM(penalty),0) penalty, COALESCE(trunc(bookings.canceled_at, 'month'), trunc(bookings.updated_at, 'month')) month, 
            city, contracts.inserted_at as sell_date, contracts.booking_id, apartments.codename,
            COALESCE(revenue.contract_id, NULL) contract_id
        
        FROM silver.revenue.revenue
            LEFT JOIN silver.backoffice.apartments ON revenue.revenue.apartment_id = backoffice.apartments.id
            LEFT JOIN silver.backoffice.contracts ON revenue.revenue.contract_id = backoffice.contracts.id
            LEFT JOIN silver.backoffice.bookings ON backoffice.contracts.booking_id = backoffice.bookings.id
            
        WHERE backoffice.bookings.state NOT IN ('canceled', 'denied')
            
        GROUP BY COALESCE(trunc(bookings.canceled_at, 'month'), trunc(bookings.updated_at, 'month')), city, COALESCE(revenue.contract_id, NULL), contracts.inserted_at, contracts.booking_id, apartments.codename
    )

    ,New AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(penalty)/1.1 
                WHEN city IN ('Berlin') THEN SUM(penalty)/1.07 
                ELSE SUM(penalty) END AS penalty, city, month, sell_date, booking_id, codename, contract_id 
        FROM NON_PRORRATED 
        --WHERE month >= trunc(current_date, 'month') - INTERVAL 12 MONTH AND month <= trunc(current_date, 'month') + INTERVAL 4 MONTH
        GROUP BY city, month, sell_date, booking_id, codename, contract_id 
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )


    SELECT city, month, sell_date, booking_id, codename, contract_id, penalty FROM New WHERE month >= DATE('2024-12-01')
    UNION ALL
    SELECT city, month, sell_date, booking_id, codename, contract_id, penalty FROM silver.finance.materialized_penalty_per_recognition
"""

spark.sql(penalty_per_recognition).write.mode("overwrite").saveAsTable("silver.finance.penalty_per_recognition")

###1.4.2. Deltas 7 Days Ago
###### (Penalty Revenue)

In [0]:
penalty_per_recognition_7_days_deltas = """
WITH NON_PRORRATED AS (
        SELECT 
            COALESCE(SUM(penalty),0) penalty, COALESCE(trunc(bookings.canceled_at, 'month'), trunc(bookings.updated_at, 'month')) month, 
            city, contracts.inserted_at as sell_date, contracts.booking_id, apartments.codename,
            COALESCE(revenue.contract_id, NULL) contract_id
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE bookings.state NOT IN ('canceled', 'denied')
            
        GROUP BY COALESCE(trunc(bookings.canceled_at, 'month'), trunc(bookings.updated_at, 'month')), city, COALESCE(revenue.contract_id, NULL), contracts.inserted_at, contracts.booking_id, apartments.codename
    )

    ,New AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(penalty)/1.1 
                WHEN city IN ('Berlin') THEN SUM(penalty)/1.07 
                ELSE SUM(penalty) END AS penalty, city, month, sell_date, booking_id, codename, contract_id 
        FROM NON_PRORRATED 
        --WHERE month >= trunc(current_date, 'month') - INTERVAL 12 MONTH AND month <= trunc(current_date, 'month') + INTERVAL 4 MONTH
        GROUP BY city, month, sell_date, booking_id, codename, contract_id 
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )


    SELECT city, month, sell_date, booking_id, codename, contract_id, penalty FROM New
"""

spark.sql(penalty_per_recognition_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.penalty_per_recognition_7_days_deltas")

###1.4.3. Deltas Since Begining of Month
###### (Penalty Revenue)

In [0]:
penalty_per_recognition_begining_month_deltas = """
WITH NON_PRORRATED AS (
        SELECT 
            COALESCE(SUM(penalty),0) penalty, COALESCE(trunc(bookings.canceled_at, 'month'), trunc(bookings.updated_at, 'month')) month, 
            city, contracts.inserted_at as sell_date, contracts.booking_id, apartments.codename,
            COALESCE(revenue.contract_id, NULL) contract_id
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE bookings.state NOT IN ('canceled', 'denied')
            
        GROUP BY COALESCE(trunc(bookings.canceled_at, 'month'), trunc(bookings.updated_at, 'month')), city, COALESCE(revenue.contract_id, NULL), contracts.inserted_at, contracts.booking_id, apartments.codename
    )

    ,New AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(penalty)/1.1 
                WHEN city IN ('Berlin') THEN SUM(penalty)/1.07 
                ELSE SUM(penalty) END AS penalty, city, month, sell_date, booking_id, codename, contract_id 
        FROM NON_PRORRATED 
        --WHERE month >= trunc(current_date, 'month') - INTERVAL 12 MONTH AND month <= trunc(current_date, 'month') + INTERVAL 4 MONTH
        GROUP BY city, month, sell_date, booking_id, codename, contract_id 
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )


    SELECT city, month, sell_date, booking_id, codename, contract_id, penalty FROM New
"""

spark.sql(penalty_per_recognition_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.penalty_per_recognition_begining_month_deltas")

##1.5. Cleaning Revenue

###1.5.1. Actuals
###### (Cleaning Revenue)

In [0]:
cleaning_per_recognition = """
WITH NON_PRORRATED AS (
        SELECT 
            COALESCE(SUM(final_cleaning),0) final_cleaning, trunc(contracts.start_date, 'month') month, city, contracts.inserted_at as sell_date, 
            contracts.booking_id, apartments.codename, COALESCE(revenue.contract_id, NULL) contract_id
        
        FROM silver.revenue.revenue
            LEFT JOIN silver.backoffice.apartments ON revenue.revenue.apartment_id = backoffice.apartments.id
            LEFT JOIN silver.backoffice.contracts ON revenue.revenue.contract_id = backoffice.contracts.id
            LEFT JOIN silver.backoffice.bookings ON backoffice.contracts.booking_id = backoffice.bookings.id
            
        WHERE city IN ('Barcelona', 'Madrid', 'Berlin')
            AND bookings.source NOT IN('Apartool', 'Silverdoor', 'Nomad', 'RelocatEU', 'Dwellworks', 'Altovita', 'Airbnb', 'HomeLike', 'VRBO')
            AND backoffice.bookings.state NOT IN ('canceled', 'denied')
        
        GROUP BY trunc(contracts.start_date, 'month'), city, contracts.inserted_at, contracts.booking_id, apartments.codename, COALESCE(revenue.contract_id, NULL) 
    )

    ,PRORRATED_5 AS (
        SELECT 
            COALESCE(SUM(final_cleaning),0) final_cleaning, city, contracts.start_date, 
            contracts.end_date, revenue.contract_id, contracts.inserted_at sell_date, 
            contracts.booking_id, apartments.codename
        
        FROM silver.revenue.revenue
            LEFT JOIN silver.backoffice.apartments ON revenue.revenue.apartment_id = backoffice.apartments.id
            LEFT JOIN silver.backoffice.contracts ON revenue.revenue.contract_id = backoffice.contracts.id
            LEFT JOIN silver.backoffice.bookings ON backoffice.contracts.booking_id = backoffice.bookings.id
            
        WHERE (city NOT IN ('Barcelona', 'Madrid', 'Berlin') OR bookings.source IN ('Apartool', 'Silverdoor', 'Nomad', 'RelocatEU', 'Dwellworks', 'Altovita', 'Airbnb', 'HomeLike', 'VRBO'))
        
        GROUP BY city, contracts.start_date, contracts.end_date, revenue.contract_id, contracts.inserted_at, contracts.booking_id, apartments.codename
    )

    ,PRORRATED_4 AS (
        SELECT 
            explode(
                sequence(
                    trunc(start_date, 'month'),
                    trunc(end_date, 'month'),
                    INTERVAL 1 MONTH
                )
            ) month, city, final_cleaning::DECIMAL / (end_date::DATE - start_date::DATE)::INTEGER AS final_cleaning, start_date check_in, end_date check_out, contract_id,
            sell_date, booking_id, codename
            
        FROM PRORRATED_5
    )

    ,PRORRATED_3 AS (
        SELECT
            *, 
            CASE
                WHEN (month) = trunc(check_out::date, 'month')::date and trunc(check_in::date, 'month') <> trunc(check_out::date, 'month') THEN EXTRACT(DAY FROM check_out::date -  trunc((month)::DATE, 'MONTH')) ::INTEGER
                WHEN (month) = trunc(check_in::date, 'month')::date and trunc(check_in::date, 'month') <> trunc(check_out::date, 'month') THEN EXTRACT(DAY FROM trunc((month + INTERVAL '1 month')::DATE, 'MONTH') - check_in)::INTEGER
                WHEN trunc(check_out::date, 'month') = trunc(check_in::date, 'month') THEN (check_out::DATE - check_in::DATE)::INTEGER
                ELSE EXTRACT(DAY FROM trunc((month + INTERVAL '1 month')::DATE, 'MONTH') - month)::INTEGER
            END as nights
            
        FROM PRORRATED_4
    )

    ,PRORRATED_2 as (
        SELECT 
            city, final_cleaning::DECIMAL * nights final_cleaning, month,sell_date, booking_id, codename, contract_id
        
        FROM PRORRATED_3
    )

    ,PRORRATED AS (
        SELECT SUM(final_cleaning) final_cleaning, city, month,sell_date, booking_id, codename, contract_id FROM PRORRATED_2 GROUP BY city, month, contract_id, sell_date, booking_id, codename
    )

    ,JOINED AS (
        SELECT city, final_cleaning, month,sell_date, booking_id, codename, contract_id FROM PRORRATED
        UNION ALL 
        SELECT city, final_cleaning, month,sell_date, booking_id, codename, contract_id FROM NON_PRORRATED
    )

    ,New AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(final_cleaning)/1.1 
                WHEN city IN ('Berlin') THEN SUM(final_cleaning)/1.07 
                ELSE SUM(final_cleaning) END AS final_cleaning, city, month,sell_date, booking_id, codename, contract_id
        FROM JOINED 
        --WHERE month >= trunc(current_date, 'month') - INTERVAL 12 MONTH AND month <= trunc(current_date, 'month') + INTERVAL 4 MONTH
        GROUP BY city, month,sell_date, booking_id, codename, contract_id 
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )

    SELECT city, month, sell_date, booking_id, codename, contract_id, final_cleaning FROM New WHERE month >= DATE('2024-12-01')
    UNION ALL
    SELECT city, month, sell_date, booking_id, codename, contract_id, final_cleaning FROM silver.finance.materialized_cleaning_per_recognition
"""

spark.sql(cleaning_per_recognition).write.mode("overwrite").saveAsTable("silver.finance.cleaning_per_recognition")

###1.5.2. Deltas 7 Days Ago
###### (Cleaning Revenue)

In [0]:
cleaning_per_recognition_7_days_deltas = """
WITH NON_PRORRATED AS (
        SELECT 
            COALESCE(SUM(final_cleaning),0) final_cleaning, trunc(contracts.start_date, 'month') month, city, contracts.inserted_at as sell_date, 
            contracts.booking_id, apartments.codename, COALESCE(revenue.contract_id, NULL) contract_id
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE city IN ('Barcelona', 'Madrid', 'Berlin')
            AND bookings.source NOT IN('Apartool', 'Silverdoor', 'Nomad', 'RelocatEU', 'Dwellworks', 'Altovita', 'Airbnb', 'HomeLike', 'VRBO')
            AND bookings.state NOT IN ('canceled', 'denied')
        
        GROUP BY trunc(contracts.start_date, 'month'), city, contracts.inserted_at, contracts.booking_id, apartments.codename, COALESCE(revenue.contract_id, NULL) 
    )

    ,PRORRATED_5 AS (
        SELECT 
            COALESCE(SUM(final_cleaning),0) final_cleaning, city, contracts.start_date, 
            contracts.end_date, revenue.contract_id, contracts.inserted_at sell_date, 
            contracts.booking_id, apartments.codename
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE (city NOT IN ('Barcelona', 'Madrid', 'Berlin') OR bookings.source IN ('Apartool', 'Silverdoor', 'Nomad', 'RelocatEU', 'Dwellworks', 'Altovita', 'Airbnb', 'HomeLike', 'VRBO'))
        
        GROUP BY city, contracts.start_date, contracts.end_date, revenue.contract_id, contracts.inserted_at, contracts.booking_id, apartments.codename
    )

    ,PRORRATED_4 AS (
        SELECT 
            explode(
                sequence(
                    trunc(start_date, 'month'),
                    trunc(end_date, 'month'),
                    INTERVAL 1 MONTH
                )
            ) month, city, final_cleaning::DECIMAL / (end_date::DATE - start_date::DATE)::INTEGER AS final_cleaning, start_date check_in, end_date check_out, contract_id,
            sell_date, booking_id, codename
            
        FROM PRORRATED_5
    )

    ,PRORRATED_3 AS (
        SELECT
            *, 
            CASE
                WHEN (month) = trunc(check_out::date, 'month')::date and trunc(check_in::date, 'month') <> trunc(check_out::date, 'month') THEN EXTRACT(DAY FROM check_out::date -  trunc((month)::DATE, 'MONTH')) ::INTEGER
                WHEN (month) = trunc(check_in::date, 'month')::date and trunc(check_in::date, 'month') <> trunc(check_out::date, 'month') THEN EXTRACT(DAY FROM trunc((month + INTERVAL '1 month')::DATE, 'MONTH') - check_in)::INTEGER
                WHEN trunc(check_out::date, 'month') = trunc(check_in::date, 'month') THEN (check_out::DATE - check_in::DATE)::INTEGER
                ELSE EXTRACT(DAY FROM trunc((month + INTERVAL '1 month')::DATE, 'MONTH') - month)::INTEGER
            END as nights
            
        FROM PRORRATED_4
    )

    ,PRORRATED_2 as (
        SELECT 
            city, final_cleaning::DECIMAL * nights final_cleaning, month,sell_date, booking_id, codename, contract_id
        
        FROM PRORRATED_3
    )

    ,PRORRATED AS (
        SELECT SUM(final_cleaning) final_cleaning, city, month,sell_date, booking_id, codename, contract_id FROM PRORRATED_2 GROUP BY city, month, contract_id, sell_date, booking_id, codename
    )

    ,JOINED AS (
        SELECT city, final_cleaning, month,sell_date, booking_id, codename, contract_id FROM PRORRATED
        UNION ALL 
        SELECT city, final_cleaning, month,sell_date, booking_id, codename, contract_id FROM NON_PRORRATED
    )

    ,New AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(final_cleaning)/1.1 
                WHEN city IN ('Berlin') THEN SUM(final_cleaning)/1.07 
                ELSE SUM(final_cleaning) END AS final_cleaning, city, month,sell_date, booking_id, codename, contract_id
        FROM JOINED 
        --WHERE month >= trunc(current_date, 'month') - INTERVAL 12 MONTH AND month <= trunc(current_date, 'month') + INTERVAL 4 MONTH
        GROUP BY city, month,sell_date, booking_id, codename, contract_id 
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )

    SELECT city, month, sell_date, booking_id, codename, contract_id, final_cleaning FROM New
"""

spark.sql(cleaning_per_recognition_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.cleaning_per_recognition_7_days_deltas")

###1.5.3. Deltas Since Begining of Month
###### (Cleaning Revenue)

In [0]:
cleaning_per_recognition_begining_month_deltas = """
WITH NON_PRORRATED AS (
        SELECT 
            COALESCE(SUM(final_cleaning),0) final_cleaning, trunc(contracts.start_date, 'month') month, city, contracts.inserted_at as sell_date, 
            contracts.booking_id, apartments.codename, COALESCE(revenue.contract_id, NULL) contract_id
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE city IN ('Barcelona', 'Madrid', 'Berlin')
            AND bookings.source NOT IN('Apartool', 'Silverdoor', 'Nomad', 'RelocatEU', 'Dwellworks', 'Altovita', 'Airbnb', 'HomeLike', 'VRBO')
            AND bookings.state NOT IN ('canceled', 'denied')
        
        GROUP BY trunc(contracts.start_date, 'month'), city, contracts.inserted_at, contracts.booking_id, apartments.codename, COALESCE(revenue.contract_id, NULL) 
    )

    ,PRORRATED_5 AS (
        SELECT 
            COALESCE(SUM(final_cleaning),0) final_cleaning, city, contracts.start_date, 
            contracts.end_date, revenue.contract_id, contracts.inserted_at sell_date, 
            contracts.booking_id, apartments.codename
        
        FROM silver.revenue.revenue TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) revenue
            LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON revenue.apartment_id = apartments.id
            LEFT JOIN silver.backoffice.contracts TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) contracts ON revenue.contract_id = contracts.id
            LEFT JOIN silver.backoffice.bookings TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) bookings ON contracts.booking_id = bookings.id
            
        WHERE (city NOT IN ('Barcelona', 'Madrid', 'Berlin') OR bookings.source IN ('Apartool', 'Silverdoor', 'Nomad', 'RelocatEU', 'Dwellworks', 'Altovita', 'Airbnb', 'HomeLike', 'VRBO'))
        
        GROUP BY city, contracts.start_date, contracts.end_date, revenue.contract_id, contracts.inserted_at, contracts.booking_id, apartments.codename
    )

    ,PRORRATED_4 AS (
        SELECT 
            explode(
                sequence(
                    trunc(start_date, 'month'),
                    trunc(end_date, 'month'),
                    INTERVAL 1 MONTH
                )
            ) month, city, final_cleaning::DECIMAL / (end_date::DATE - start_date::DATE)::INTEGER AS final_cleaning, start_date check_in, end_date check_out, contract_id,
            sell_date, booking_id, codename
            
        FROM PRORRATED_5
    )

    ,PRORRATED_3 AS (
        SELECT
            *, 
            CASE
                WHEN (month) = trunc(check_out::date, 'month')::date and trunc(check_in::date, 'month') <> trunc(check_out::date, 'month') THEN EXTRACT(DAY FROM check_out::date -  trunc((month)::DATE, 'MONTH')) ::INTEGER
                WHEN (month) = trunc(check_in::date, 'month')::date and trunc(check_in::date, 'month') <> trunc(check_out::date, 'month') THEN EXTRACT(DAY FROM trunc((month + INTERVAL '1 month')::DATE, 'MONTH') - check_in)::INTEGER
                WHEN trunc(check_out::date, 'month') = trunc(check_in::date, 'month') THEN (check_out::DATE - check_in::DATE)::INTEGER
                ELSE EXTRACT(DAY FROM trunc((month + INTERVAL '1 month')::DATE, 'MONTH') - month)::INTEGER
            END as nights
            
        FROM PRORRATED_4
    )

    ,PRORRATED_2 as (
        SELECT 
            city, final_cleaning::DECIMAL * nights final_cleaning, month,sell_date, booking_id, codename, contract_id
        
        FROM PRORRATED_3
    )

    ,PRORRATED AS (
        SELECT SUM(final_cleaning) final_cleaning, city, month,sell_date, booking_id, codename, contract_id FROM PRORRATED_2 GROUP BY city, month, contract_id, sell_date, booking_id, codename
    )

    ,JOINED AS (
        SELECT city, final_cleaning, month,sell_date, booking_id, codename, contract_id FROM PRORRATED
        UNION ALL 
        SELECT city, final_cleaning, month,sell_date, booking_id, codename, contract_id FROM NON_PRORRATED
    )

    ,New AS (
        SELECT 
            CASE WHEN city IN ('Barcelona', 'Madrid') THEN SUM(final_cleaning)/1.1 
                WHEN city IN ('Berlin') THEN SUM(final_cleaning)/1.07 
                ELSE SUM(final_cleaning) END AS final_cleaning, city, month,sell_date, booking_id, codename, contract_id
        FROM JOINED 
        --WHERE month >= trunc(current_date, 'month') - INTERVAL 12 MONTH AND month <= trunc(current_date, 'month') + INTERVAL 4 MONTH
        GROUP BY city, month,sell_date, booking_id, codename, contract_id 
        ORDER BY month ASC, CASE WHEN city = 'Barcelona' THEN 1
                                WHEN city = 'Madrid' THEN 2
                                WHEN city = 'Lisbon' THEN 3
                                WHEN city = 'Paris' THEN 4
                                WHEN city = 'Berlin' THEN 5
                            END ASC
    )

    SELECT city, month, sell_date, booking_id, codename, contract_id, final_cleaning FROM New
"""

spark.sql(cleaning_per_recognition_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.cleaning_per_recognition_begining_month_deltas")

##1.6. Joined Revenue
###### (Actuals / 7 Days Ago / Since Beginning of Month)

In [0]:
############################################################################################################
gross_revenue = """  
    WITH X AS (
      SELECT sell_date, city, codename, penalty revenue, month, booking_id, 'Revenue - Other Income (€)' management_accounts, contract_id FROM silver.finance.penalty_per_recognition
      UNION ALL
      SELECT sell_date, city, codename, Utilities revenue, month, booking_id, 'Revenue - Utilities (€)' management_accounts, contract_id FROM silver.finance.utilities_per_recognition
      UNION ALL
      SELECT sell_date, city, codename, service_charge revenue, month, booking_id, 'Revenue - Service Charge (€)' management_accounts, contract_id FROM silver.finance.sc_per_recognition
      UNION ALL
      SELECT sell_date, city, codename, rent revenue, month, booking_id, 'Revenue - Rental (€)' management_accounts, contract_id FROM silver.finance.rent_per_recognition
      UNION ALL
      SELECT sell_date, city, codename, final_cleaning revenue, month, booking_id, 'Revenue - Cleaning Services (€)' management_accounts, contract_id FROM silver.finance.cleaning_per_recognition
  )

  ,Sales AS (
      SELECT * FROM silver.finance.new_sales_and_source
  )

  , A AS (
      SELECT X.*, Sales.type FROM X LEFT JOIN Sales ON X.contract_id = Sales.contract_id
  )

  SELECT * FROM A
"""
spark.sql(gross_revenue).write.mode("overwrite").saveAsTable("silver.finance.gross_revenue")

############################################################################################################
actuals_fcst_gross_revenue = """  
    WITH fcst AS (
    SELECT SUM(revenue) amount, management_accounts, month, city
    FROM silver.finance.gross_revenue
    GROUP BY management_accounts, month, city
    )

    ,Actuals AS (
    SELECT SUM(amount) amount, `Management Accounts` management_accounts, city, to_date(month, 'd/M/yyyy') as month
    FROM silver.finance.gm_by_apt_oct_24
    WHERE `Line Type` = 'Revenue'
    GROUP BY `Management Accounts`, city, month
    )

    ,Series AS (
    SELECT DISTINCT management_accounts, city, explode(sequence(DATE('2023-01-01'), trunc(current_date, 'month') + INTERVAL 24 MONTH, INTERVAL 1 MONTH)) month,NULL amount
    FROM Actuals
    )

    SELECT COALESCE(SUM(actuals.amount), SUM(fcst.amount)) amount, Series.month, Series.management_accounts, Series.city
    FROM Series 
    LEFT JOIN fcst ON fcst.month = Series.month AND fcst.management_accounts = Series.management_accounts AND fcst.city = Series.city
    LEFT JOIN actuals ON Series.month = actuals.month AND Series.management_accounts = actuals.management_accounts AND Series.city = actuals.city
    GROUP BY Series.month, Series.management_accounts, Series.city
    ORDER BY Series.month DESC, Series.management_accounts desc, Series.city DESC
"""
spark.sql(actuals_fcst_gross_revenue).write.mode("overwrite").saveAsTable("silver.finance.actuals_fcst_gross_revenue")

############################################################################################################
gross_revenue_7_days_deltas = """  
    WITH X AS (
      SELECT sell_date, city, codename, penalty revenue, month, booking_id, 'Revenue - Other Income (€)' management_accounts, contract_id FROM silver.finance.penalty_per_recognition_7_days_deltas
      UNION ALL
      SELECT sell_date, city, codename, Utilities revenue, month, booking_id, 'Revenue - Utilities (€)' management_accounts, contract_id FROM silver.finance.utilities_per_recognition_7_days_deltas
      UNION ALL
      SELECT sell_date, city, codename, service_charge revenue, month, booking_id, 'Revenue - Service Charge (€)' management_accounts, contract_id FROM silver.finance.sc_per_recognition_7_days_deltas
      UNION ALL
      SELECT sell_date, city, codename, rent revenue, month, booking_id, 'Revenue - Rental (€)' management_accounts, contract_id FROM silver.finance.rent_per_recognition_7_days_deltas
      UNION ALL
      SELECT sell_date, city, codename, final_cleaning revenue, month, booking_id, 'Revenue - Cleaning Services (€)' management_accounts, contract_id FROM silver.finance.cleaning_per_recognition_7_days_deltas
  )

  ,Sales AS (
      SELECT * FROM silver.finance.new_sales_and_source_7_days_deltas
  )

  , A AS (
      SELECT X.*, Sales.type FROM X LEFT JOIN Sales ON X.contract_id = Sales.contract_id
  )

  SELECT * FROM A
"""
spark.sql(gross_revenue_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.gross_revenue_7_days_deltas")

############################################################################################################
gross_revenue_begining_month_deltas = """  
    WITH X AS (
      SELECT sell_date, city, codename, penalty revenue, month, booking_id, 'Revenue - Other Income (€)' management_accounts, contract_id FROM silver.finance.penalty_per_recognition_begining_month_deltas
      UNION ALL
      SELECT sell_date, city, codename, Utilities revenue, month, booking_id, 'Revenue - Utilities (€)' management_accounts, contract_id FROM silver.finance.utilities_per_recognition_begining_month_deltas
      UNION ALL
      SELECT sell_date, city, codename, service_charge revenue, month, booking_id, 'Revenue - Service Charge (€)' management_accounts, contract_id FROM silver.finance.sc_per_recognition_begining_month_deltas
      UNION ALL
      SELECT sell_date, city, codename, rent revenue, month, booking_id, 'Revenue - Rental (€)' management_accounts, contract_id FROM silver.finance.rent_per_recognition_begining_month_deltas
      UNION ALL
      SELECT sell_date, city, codename, final_cleaning revenue, month, booking_id, 'Revenue - Cleaning Services (€)' management_accounts, contract_id FROM silver.finance.cleaning_per_recognition_begining_month_deltas
  )

  ,Sales AS (
      SELECT * FROM silver.finance.new_sales_and_source_begining_month_deltas
  )

  , A AS (
      SELECT X.*, Sales.type FROM X LEFT JOIN Sales ON X.contract_id = Sales.contract_id
  )

  SELECT * FROM A
"""
spark.sql(gross_revenue_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.gross_revenue_begining_month_deltas")


#

# 2. Revenue per Sell Date

###2.1. Actuals
###### (Revenue per Sell Date)

In [0]:
revenue_per_sell_date = """
WITH X AS (
        SELECT sell_date, city, codename, penalty revenue, month, booking_id, 'Penalty' tipo, contract_id FROM silver.finance.penalty_per_recognition
        UNION ALL
        SELECT sell_date, city, codename, Utilities revenue, month, booking_id, 'Utilities' tipo, contract_id FROM silver.finance.utilities_per_recognition
        UNION ALL
        SELECT sell_date, city, codename, service_charge revenue, month, booking_id, 'Service Charge' tipo, contract_id FROM silver.finance.sc_per_recognition
        UNION ALL
        SELECT sell_date, city, codename, rent revenue, month, booking_id, 'Rent' tipo, contract_id FROM silver.finance.rent_per_recognition
        UNION ALL
        SELECT sell_date, city, codename, final_cleaning revenue, month, booking_id, 'Final Cleaning' tipo, contract_id FROM silver.finance.cleaning_per_recognition
    )

    SELECT SUM(revenue) revenue, tipo, trunc(sell_date, 'month') month FROM X GROUP BY tipo, trunc(sell_date, 'month')
"""

spark.sql(revenue_per_sell_date).write.mode("overwrite").saveAsTable("silver.finance.revenue_per_sell_date")

###2.2. Deltas 7 Days Ago
###### (Revenue per Sell Date)

In [0]:
revenue_per_sell_date_7_days_deltas = """
WITH X AS (
        SELECT sell_date, city, codename, penalty revenue, month, booking_id, 'Penalty' tipo, contract_id FROM silver.finance.penalty_per_recognition_7_days_deltas
        UNION ALL
        SELECT sell_date, city, codename, Utilities revenue, month, booking_id, 'Utilities' tipo, contract_id FROM silver.finance.utilities_per_recognition_7_days_deltas
        UNION ALL
        SELECT sell_date, city, codename, service_charge revenue, month, booking_id, 'Service Charge' tipo, contract_id FROM silver.finance.sc_per_recognition_7_days_deltas
        UNION ALL
        SELECT sell_date, city, codename, rent revenue, month, booking_id, 'Rent' tipo, contract_id FROM silver.finance.rent_per_recognition_7_days_deltas
        UNION ALL
        SELECT sell_date, city, codename, final_cleaning revenue, month, booking_id, 'Final Cleaning' tipo, contract_id FROM silver.finance.cleaning_per_recognition_7_days_deltas
    )

    SELECT SUM(revenue) revenue, tipo, trunc(sell_date, 'month') month FROM X GROUP BY tipo, trunc(sell_date, 'month')
"""

spark.sql(revenue_per_sell_date_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.revenue_per_sell_date_7_days_deltas")

###2.3. Deltas Since Begining of Month
###### (Revenue per Sell Date)

In [0]:
revenue_per_sell_date_begining_month_deltas = """
WITH X AS (
        SELECT sell_date, city, codename, penalty revenue, month, booking_id, 'Penalty' tipo, contract_id FROM silver.finance.penalty_per_recognition_begining_month_deltas
        UNION ALL
        SELECT sell_date, city, codename, Utilities revenue, month, booking_id, 'Utilities' tipo, contract_id FROM silver.finance.utilities_per_recognition_begining_month_deltas
        UNION ALL
        SELECT sell_date, city, codename, service_charge revenue, month, booking_id, 'Service Charge' tipo, contract_id FROM silver.finance.sc_per_recognition_begining_month_deltas
        UNION ALL
        SELECT sell_date, city, codename, rent revenue, month, booking_id, 'Rent' tipo, contract_id FROM silver.finance.rent_per_recognition_begining_month_deltas
        UNION ALL
        SELECT sell_date, city, codename, final_cleaning revenue, month, booking_id, 'Final Cleaning' tipo, contract_id FROM silver.finance.cleaning_per_recognition_begining_month_deltas
    )

    SELECT SUM(revenue) revenue, tipo, trunc(sell_date, 'month') month FROM X GROUP BY tipo, trunc(sell_date, 'month')
"""

spark.sql(revenue_per_sell_date_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.revenue_per_sell_date_begining_month_deltas")

#

# 3. COGS

##3.1. COGS Forecast (Utilities & Cleaning)

### 3.1.1.Forecast Slopes

In [0]:
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

############################################################################################################
############################################################################################################
#SCRIPT
bookable = spark.table("silver.finance.bookable_and_booked")
bookable = bookable.toPandas()
apts = spark.table("silver.backoffice.apartments")
apts = apts.toPandas()
volume = spark.table("silver.finance.volume_database")
volume = volume.toPandas()
gm_by_apt = spark.table("silver.finance.gm_by_apt_oct_24")
gm_by_apt = gm_by_apt.toPandas()

volume = volume.rename(columns={'month_date': 'month'})

volume['month'] = pd.to_datetime(volume['month'], format='%Y-%m-%d')
bookable['month'] = pd.to_datetime(bookable['month'], format='%Y-%m-%d')
############################################################################################################
############################################################################################################

gm_by_apt['Month'] = pd.to_datetime(gm_by_apt['Month'], format='%d/%m/%Y')

############################################################################################################
############################################################################################################

cleaning_services = (
    gm_by_apt[gm_by_apt['Management Accounts'] == 'COGS - Cleaning Services (€)']
    .groupby(['City', 'Month'], as_index=False)
    .agg(Cleaning=('Amount', 'sum'))
)

utilities_services = (
    gm_by_apt[gm_by_apt['Management Accounts'] == 'COGS - Utilities (€)']
    .groupby(['City', 'Month'], as_index=False)
    .agg(Utilities=('Amount', 'sum'))
)

############################################################################################################
############################################################################################################

cogs = pd.merge(cleaning_services, utilities_services, on=['City', 'Month'], how='left')

bookable = pd.merge(bookable, apts[['id', 'city', 'total_sqm','max_guests']], left_on='apartment_id', right_on='id',how='left')

############################################################################################################
############################################################################################################

bookable = pd.merge(bookable, volume[['month', 'id', 'check_out']], on=['month', 'id'], how='left')

booked = (
    bookable.groupby(['city', 'month'], as_index=False)
    .agg(Booked=('booked', 'sum'))
)

check_out = (
    bookable.groupby(['city', 'month'], as_index=False)
    .agg(Check_out=('check_out', 'sum'))
)

bookable_2 = pd.merge(booked, check_out, on=['city', 'month'], how='left')
bookable_2['month'] = pd.to_datetime(bookable_2['month'], format='%Y-%m-%d')
bookable_2 = bookable_2.rename(columns={'city': 'City', 'month': 'Month'})

raw_forecast = pd.merge(bookable_2, cogs, on=['City', 'Month'], how='left')

############################################################################################################
############################################################################################################

current_date = pd.Timestamp.now()

start_date = (current_date - pd.DateOffset(months=12)).normalize()
end_date = current_date.normalize() - pd.DateOffset(months=2)

raw_forecast = raw_forecast[(raw_forecast['Month'] >= start_date) & (raw_forecast['Month'] < end_date)]

raw_forecast = raw_forecast[(raw_forecast['Month'] != '09-01-2024')]

future_forecast = bookable_2[(bookable_2['Month'] >= end_date)]

############################################################################################################
############################################################################################################

raw_forecast.sort_values(by=['Month', 'City'], ascending=[True, True], inplace=True)
raw_forecast.reset_index(drop=True, inplace=True)

############################################################################################################
############################################################################################################

enc = OneHotEncoder(sparse_output=False)  # Asegura que devuelve un array denso
encoded_columns = enc.fit_transform(raw_forecast[['City']])

encoded_column_names = enc.get_feature_names_out(['City'])

assert encoded_columns.shape[1] == len(encoded_column_names), "Las dimensiones no coinciden."

encoded_df = pd.DataFrame(encoded_columns, columns=encoded_column_names)

encoded_df = pd.DataFrame(encoded_df)  # Convert list to DataFrame
raw_forecast = pd.DataFrame(raw_forecast)  # Convert list to DataFrame

raw_forecast = pd.merge(raw_forecast, encoded_df, left_index=True, right_index=True)

raw_forecast.rename(columns={'City_Barcelona': 'Barcelona', 'City_Berlin': 'Berlin',
                              'City_Lisbon': 'Lisbon', 'City_Madrid': 'Madrid', 'City_Paris': 'Paris'}, inplace=True)

raw_forecast = raw_forecast.drop(['City'], axis=1)

############################################################################################################
############################################################################################################

X = raw_forecast[['Check_out', 'Berlin', 'Lisbon', 'Madrid', 'Paris', 'Barcelona']]
y = raw_forecast['Cleaning']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

coefficients = model.coef_

feature_names = X.columns

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

coefficients_fc = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})

coefficients_fc = coefficients_fc.append({'Feature': 'Intercept', 'Coefficient': model.intercept_}, ignore_index=True)

############################################################################################################
############################################################################################################

X = raw_forecast[['Booked', 'Berlin', 'Lisbon', 'Madrid', 'Paris', 'Barcelona']]
y = raw_forecast['Utilities']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

coefficients = model.coef_

feature_names = X.columns

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

coefficients_u = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})

coefficients_u = coefficients_u.append({'Feature': 'Intercept', 'Coefficient': model.intercept_}, ignore_index=True)

############################################################################################################
############################################################################################################

replace_1 = {'Berlin': 'Berlin_Utilities',
            'Lisbon': 'Lisbon_Utilities',
            'Madrid': 'Madrid_Utilities',
            'Paris': 'Paris_Utilities',
            'Barcelona': 'Barcelona_Utilities',
            'Intercept': 'Intercept_Utilities'}

coefficients_u = coefficients_u.replace(replace_1)

replace_2 = {'Berlin': 'Berlin_FC',
            'Lisbon': 'Lisbon_FC',
            'Madrid': 'Madrid_FC',
            'Paris': 'Paris_FC',
            'Barcelona': 'Barcelona_FC',
            'Intercept': 'Intercept_FC'}

coefficients_u_raw = coefficients_u.replace(replace_1)
coefficients_fc_raw = coefficients_fc.replace(replace_2)

############################################################################################################
############################################################################################################

coefficients_u = spark.createDataFrame(coefficients_u_raw)
coefficients_fc = spark.createDataFrame(coefficients_fc_raw)

slopes = coefficients_u.union(coefficients_fc)

############################################################################################################
############################################################################################################

slopes.write.mode("overwrite").saveAsTable("silver.finance.COGS_Forecast_Slopes")
slopes.show()
slopes.printSchema()

/home/spark-3cb9dbb9-004a-4d7a-9877-7b/.ipykernel/8742/command-7275607494463741-1552780637:138: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coefficients_fc = coefficients_fc.append({'Feature': 'Intercept', 'Coefficient': model.intercept_}, ignore_index=True)
/home/spark-3cb9dbb9-004a-4d7a-9877-7b/.ipykernel/8742/command-7275607494463741-1552780637:165: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coefficients_u = coefficients_u.append({'Feature': 'Intercept', 'Coefficient': model.intercept_}, ignore_index=True)


+-------------------+-------------------+
|            Feature|        Coefficient|
+-------------------+-------------------+
|             Booked| -3.991618313332915|
|   Berlin_Utilities| -5223.486246489196|
|   Lisbon_Utilities|  -32112.2996042457|
|   Madrid_Utilities| -8619.158473407335|
|    Paris_Utilities|-10994.280630988655|
|Barcelona_Utilities| -5776.340504804001|
|Intercept_Utilities|                0.0|
|          Check_out| -128.4637318572615|
|          Berlin_FC| -1114.018033386601|
|          Lisbon_FC|-2013.6368496348325|
|          Madrid_FC| -24219.21617255906|
|           Paris_FC|-6787.1413052206135|
|       Barcelona_FC| -36475.54426920587|
|       Intercept_FC|                0.0|
+-------------------+-------------------+

root
 |-- Feature: string (nullable = true)
 |-- Coefficient: double (nullable = true)



###3.1.2. COGS Forecast Table

#### 3.1.2.1. Actuals
###### (COGS Forecast Table)

In [0]:
cogs_forecast_utilities_fc = """
WITH Volume AS (
        SELECT * FROM silver.finance.volume_database
    )

    ,Ttl_apts AS (
        SELECT SUM(guest_ready) apts, month_date, city FROM volume GROUP BY month_date, city
    )

    ---------------------------------------------------------------------------------------------------
    --------------------------------- UTILITIES FORECAST ----------------------------------------------
    ---------------------------------------------------------------------------------------------------

    ,Slopes_Utilities_1 AS (
        SELECT *, explode(split(Feature,'_')) feature_2 FROM silver.finance.cogs_forecast_slopes WHERE (Feature ILIKE '%utilities%' OR Feature = 'Booked')
    )

    ,Slopes_Utilities AS (
        SELECT Coefficient, feature_2 feature FROM Slopes_Utilities_1 WHERE feature_2 <> 'Utilities'
    )

    ,booked AS (
        SELECT SUM(occupied_nights) booked, month_date month, city, codename FROM volume WHERE month_date >= DATE('2024-01-01') GROUP BY month_date, city, codename ORDER BY month_date DESC
    )

    , booked_Fcst AS (
        SELECT 
            b.month, b.city, b.codename,
            CASE 
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Barcelona' THEN (b.booked::DECIMAL * -4.10)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Barcelona' THEN (b.booked::DECIMAL * -6.10)
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Madrid' THEN (b.booked::DECIMAL * -4.87)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Madrid' THEN (b.booked::DECIMAL * -5.10)
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Lisbon' THEN (b.booked::DECIMAL * -9.23)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Lisbon' THEN (b.booked::DECIMAL * -9.42)
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Berlin' THEN (b.booked::DECIMAL * -4.10)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Berlin' THEN (b.booked::DECIMAL * -7.02)
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Paris' THEN (b.booked::DECIMAL * -4.10)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Paris' THEN (b.booked::DECIMAL * -5.43)
            ELSE ((b.booked::DECIMAL * (uu.Coefficient)) + (uuu.Coefficient/t.apts) + COALESCE((u.Coefficient/t.apts),0)) 
            END AS amount

        FROM booked b 
            LEFT JOIN Slopes_Utilities u ON u.feature = b.city
            LEFT JOIN Ttl_apts t ON t.city = b.city AND t.month_date = b.month
            CROSS JOIN Slopes_Utilities uu 
            CROSS JOIN Slopes_Utilities uuu  
        
        WHERE uu.feature = 'Booked' AND uuu.feature = 'Intercept'
    )

    ---------------------------------------------------------------------------------------------------
    ---------------------------------- CLEANING FORECAST ----------------------------------------------
    ---------------------------------------------------------------------------------------------------

    ,Slopes_FC_1 AS (
        SELECT *, explode(split(Feature,'_')) feature_2 FROM silver.finance.cogs_forecast_slopes WHERE (Feature ILIKE '%fc%' OR Feature = 'Check_out')
    )

    ,Slopes_FC AS (
        SELECT Coefficient, feature_2 feature FROM Slopes_FC_1 WHERE feature_2 NOT IN ('FC', 'out')
    )

    ,Check_Out AS (
        SELECT SUM(check_out) check_out, month_date month, city, codename FROM volume WHERE month_date >= DATE('2024-01-01') GROUP BY month_date, city, codename ORDER BY month_date DESC
    )

    ,Cleaning_Fcst AS (
        SELECT 
            c.month, c.city, c.codename, c.Check_Out,
            CASE 
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Barcelona' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Barcelona' THEN (c.Check_Out::DECIMAL * -420)
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Madrid' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Madrid' THEN (c.Check_Out::DECIMAL * -470)
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Lisbon' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Lisbon' THEN (c.Check_Out::DECIMAL * -110)
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Berlin' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Berlin' THEN (c.Check_Out::DECIMAL * -213)
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Paris' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Paris' THEN (c.Check_Out::DECIMAL * -229)
            ELSE ((c.Check_Out::DECIMAL * (ss.Coefficient)) + (sss.Coefficient/t.apts) + COALESCE((s.Coefficient/t.apts),0)) 
            END AS amount

        FROM Check_Out c 
            LEFT JOIN Slopes_FC s ON s.feature = c.city
            LEFT JOIN Ttl_apts t ON t.city = c.city AND t.month_date = c.month
            CROSS JOIN Slopes_FC ss 
            CROSS JOIN Slopes_FC sss  
        
        WHERE ss.feature = 'Check' AND sss.feature = 'Intercept'
    )

    SELECT amount, city, codename, month, 'COGS - Cleaning Services (€)' management_accounts FROM Cleaning_Fcst
    UNION ALL
    SELECT amount, city, codename, month, 'COGS - Utilities (€)' management_accounts FROM booked_Fcst ORDER BY month, city, codename DESC
"""

spark.sql(cogs_forecast_utilities_fc).write.mode("overwrite").saveAsTable("silver.finance.cogs_forecast_utilities_fc")


#### 3.1.2.2. Deltas 7 Days Ago
###### (COGS Forecast Table)

In [0]:
cogs_forecast_utilities_fc_7_days_deltas = """
WITH Volume AS (
        SELECT * FROM silver.finance.volume_database_7_days_deltas
    )

    ,Ttl_apts AS (
        SELECT SUM(guest_ready) apts, month_date, city FROM volume GROUP BY month_date, city
    )

    ---------------------------------------------------------------------------------------------------
    --------------------------------- UTILITIES FORECAST ----------------------------------------------
    ---------------------------------------------------------------------------------------------------

    ,Slopes_Utilities_1 AS (
        SELECT *, explode(split(Feature,'_')) feature_2 FROM silver.finance.cogs_forecast_slopes WHERE (Feature ILIKE '%utilities%' OR Feature = 'Booked')
    )

    ,Slopes_Utilities AS (
        SELECT Coefficient, feature_2 feature FROM Slopes_Utilities_1 WHERE feature_2 <> 'Utilities'
    )

    ,booked AS (
        SELECT SUM(occupied_nights) booked, month_date month, city, codename FROM volume WHERE month_date >= DATE('2024-01-01') GROUP BY month_date, city, codename ORDER BY month_date DESC
    )

    , booked_Fcst AS (
        SELECT 
            b.month, b.city, b.codename, 
            CASE 
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Barcelona' THEN (b.booked::DECIMAL * -4.10)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Barcelona' THEN (b.booked::DECIMAL * -6.10)
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Madrid' THEN (b.booked::DECIMAL * -4.87)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Madrid' THEN (b.booked::DECIMAL * -5.10)
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Lisbon' THEN (b.booked::DECIMAL * -9.23)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Lisbon' THEN (b.booked::DECIMAL * -9.42)
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Berlin' THEN (b.booked::DECIMAL * -4.10)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Berlin' THEN (b.booked::DECIMAL * -7.02)
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Paris' THEN (b.booked::DECIMAL * -4.10)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Paris' THEN (b.booked::DECIMAL * -5.43)
            ELSE ((b.booked::DECIMAL * (uu.Coefficient)) + (uuu.Coefficient/t.apts) + COALESCE((u.Coefficient/t.apts),0)) 
            END AS amount

        FROM booked b 
            LEFT JOIN Slopes_Utilities u ON u.feature = b.city
            LEFT JOIN Ttl_apts t ON t.city = b.city AND t.month_date = b.month
            CROSS JOIN Slopes_Utilities uu 
            CROSS JOIN Slopes_Utilities uuu  
        
        WHERE uu.feature = 'Booked' AND uuu.feature = 'Intercept'
    )

    ---------------------------------------------------------------------------------------------------
    ---------------------------------- CLEANING FORECAST ----------------------------------------------
    ---------------------------------------------------------------------------------------------------

    ,Slopes_FC_1 AS (
        SELECT *, explode(split(Feature,'_')) feature_2 FROM silver.finance.cogs_forecast_slopes WHERE (Feature ILIKE '%fc%' OR Feature = 'Check_out')
    )

    ,Slopes_FC AS (
        SELECT Coefficient, feature_2 feature FROM Slopes_FC_1 WHERE feature_2 NOT IN ('FC', 'out')
    )

    ,Check_Out AS (
        SELECT SUM(check_out) check_out, month_date month, city, codename FROM volume WHERE month_date >= DATE('2024-01-01') GROUP BY month_date, city, codename ORDER BY month_date DESC
    )

    ,Cleaning_Fcst AS (
        SELECT 
            c.month, c.city, c.codename, c.Check_Out,
            CASE 
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Barcelona' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Barcelona' THEN (c.Check_Out::DECIMAL * -420)
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Madrid' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Madrid' THEN (c.Check_Out::DECIMAL * -470)
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Lisbon' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Lisbon' THEN (c.Check_Out::DECIMAL * -110)
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Berlin' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Berlin' THEN (c.Check_Out::DECIMAL * -213)
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Paris' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Paris' THEN (c.Check_Out::DECIMAL * -229)
            ELSE ((c.Check_Out::DECIMAL * (ss.Coefficient)) + (sss.Coefficient/t.apts) + COALESCE((s.Coefficient/t.apts),0)) 
            END AS amount

        FROM Check_Out c 
            LEFT JOIN Slopes_FC s ON s.feature = c.city
            LEFT JOIN Ttl_apts t ON t.city = c.city AND t.month_date = c.month
            CROSS JOIN Slopes_FC ss 
            CROSS JOIN Slopes_FC sss  
        
        WHERE ss.feature = 'Check' AND sss.feature = 'Intercept'
    )

    SELECT amount, city, codename, month, 'COGS - Cleaning Services (€)' management_accounts FROM Cleaning_Fcst
    UNION ALL
    SELECT amount, city, codename, month, 'COGS - Utilities (€)' management_accounts FROM booked_Fcst ORDER BY month, city, codename DESC
"""

spark.sql(cogs_forecast_utilities_fc_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.cogs_forecast_utilities_fc_7_days_deltas")

#### 3.1.2.3. Deltas Since Begining of Month
###### (COGS Forecast Table)

In [0]:
cogs_forecast_utilities_begining_month_deltas = """
WITH Volume AS (
        SELECT * FROM silver.finance.volume_database_begining_month_deltas
    )

    ,Ttl_apts AS (
        SELECT SUM(guest_ready) apts, month_date, city FROM volume GROUP BY month_date, city
    )

    ---------------------------------------------------------------------------------------------------
    --------------------------------- UTILITIES FORECAST ----------------------------------------------
    ---------------------------------------------------------------------------------------------------

    ,Slopes_Utilities_1 AS (
        SELECT *, explode(split(Feature,'_')) feature_2 FROM silver.finance.cogs_forecast_slopes WHERE (Feature ILIKE '%utilities%' OR Feature = 'Booked')
    )

    ,Slopes_Utilities AS (
        SELECT Coefficient, feature_2 feature FROM Slopes_Utilities_1 WHERE feature_2 <> 'Utilities'
    )

    ,booked AS (
        SELECT SUM(occupied_nights) booked, month_date month, city, codename FROM volume WHERE month_date >= DATE('2024-01-01') GROUP BY month_date, city, codename ORDER BY month_date DESC
    )

    , booked_Fcst AS (
        SELECT 
            b.month, b.city, b.codename, 
            CASE 
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Barcelona' THEN (b.booked::DECIMAL * -4.10)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Barcelona' THEN (b.booked::DECIMAL * -6.10)
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Madrid' THEN (b.booked::DECIMAL * -4.87)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Madrid' THEN (b.booked::DECIMAL * -5.10)
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Lisbon' THEN (b.booked::DECIMAL * -9.23)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Lisbon' THEN (b.booked::DECIMAL * -9.42)
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Berlin' THEN (b.booked::DECIMAL * -4.10)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Berlin' THEN (b.booked::DECIMAL * -7.02)
            WHEN b.month = DATE('2024-12-01') AND b.city = 'Paris' THEN (b.booked::DECIMAL * -4.10)
            WHEN b.month >= DATE('2025-01-01') AND b.month <= DATE('2025-04-01') AND b.city = 'Paris' THEN (b.booked::DECIMAL * -5.43)
            ELSE ((b.booked::DECIMAL * (uu.Coefficient)) + (uuu.Coefficient/t.apts) + COALESCE((u.Coefficient/t.apts),0)) 
            END AS amount

        FROM booked b 
            LEFT JOIN Slopes_Utilities u ON u.feature = b.city
            LEFT JOIN Ttl_apts t ON t.city = b.city AND t.month_date = b.month
            CROSS JOIN Slopes_Utilities uu 
            CROSS JOIN Slopes_Utilities uuu  
        
        WHERE uu.feature = 'Booked' AND uuu.feature = 'Intercept'
    )

    ---------------------------------------------------------------------------------------------------
    ---------------------------------- CLEANING FORECAST ----------------------------------------------
    ---------------------------------------------------------------------------------------------------

    ,Slopes_FC_1 AS (
        SELECT *, explode(split(Feature,'_')) feature_2 FROM silver.finance.cogs_forecast_slopes WHERE (Feature ILIKE '%fc%' OR Feature = 'Check_out')
    )

    ,Slopes_FC AS (
        SELECT Coefficient, feature_2 feature FROM Slopes_FC_1 WHERE feature_2 NOT IN ('FC', 'out')
    )

    ,Check_Out AS (
        SELECT SUM(check_out) check_out, month_date month, city, codename FROM volume WHERE month_date >= DATE('2024-01-01') GROUP BY month_date, city, codename ORDER BY month_date DESC
    )

    ,Cleaning_Fcst AS (
        SELECT 
            c.month, c.city, c.codename, c.Check_Out,
            CASE 
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Barcelona' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Barcelona' THEN (c.Check_Out::DECIMAL * -420)
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Madrid' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Madrid' THEN (c.Check_Out::DECIMAL * -470)
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Lisbon' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Lisbon' THEN (c.Check_Out::DECIMAL * -110)
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Berlin' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Berlin' THEN (c.Check_Out::DECIMAL * -213)
            WHEN c.month = DATE('2024-12-01') AND c.city = 'Paris' THEN (c.Check_Out::DECIMAL * -317)
            WHEN c.month >= DATE('2025-01-01') AND c.month <= DATE('2025-04-01') AND c.city = 'Paris' THEN (c.Check_Out::DECIMAL * -229)
            ELSE ((c.Check_Out::DECIMAL * (ss.Coefficient)) + (sss.Coefficient/t.apts) + COALESCE((s.Coefficient/t.apts),0)) 
            END AS amount

        FROM Check_Out c 
            LEFT JOIN Slopes_FC s ON s.feature = c.city
            LEFT JOIN Ttl_apts t ON t.city = c.city AND t.month_date = c.month
            CROSS JOIN Slopes_FC ss 
            CROSS JOIN Slopes_FC sss  
        
        WHERE ss.feature = 'Check' AND sss.feature = 'Intercept'
    )

    SELECT amount, city, codename, month, 'COGS - Cleaning Services (€)' management_accounts FROM Cleaning_Fcst
    UNION ALL
    SELECT amount, city, codename, month, 'COGS - Utilities (€)' management_accounts FROM booked_Fcst ORDER BY month, city, codename DESC
"""

spark.sql(cogs_forecast_utilities_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.cogs_forecast_utilities_begining_month_deltas")

## 3.2. COGS Per month & city

### 3.2.1. Actuals
###### (COGS Forecast Table)

In [0]:
cogs_per_month_and_city_per_codename = """
WITH COGS AS (
        SELECT sum(amount)*-1 amount, to_date(month, 'd/M/yyyy') month, `Management Accounts` AS management_accounts, city, Apartment2 codename 
        FROM silver.finance.gm_by_apt_oct_24 
        WHERE `Line Type` = 'COGS' GROUP BY to_date(month, 'd/M/yyyy'), `Management Accounts`, city, Apartment2
    )

    ,Additional_costs_2 AS (
        SELECT
            ams_contract_terms.apartment_id, city, JSON_TUPLE(RENT_INFO, 'additional_costs_amount') AS additional_costs_amount
        
        FROM silver.backoffice.ams_contract_terms 
            LEFT JOIN silver.Backoffice.apartments ON apartments.id = ams_contract_terms.apartment_id
    )

    ,Backoffice_3 AS (
        SELECT 
        SUM(landlord_daily_rent)::decimal as amount, 
        trunc(day, 'month') as month, 'COGS - Rent (€)' AS management_accounts, city, 
        landlord.apartment_id, landlord.end_of_grace_period
        FROM silver.revenue.landlord LEFT JOIN silver.Backoffice.apartments ON apartments.id = landlord.apartment_id
        WHERE trunc(day, 'month') >= trunc(current_date, 'month') - INTERVAL '1 MONTH'
        GROUP BY trunc(day, 'month'), city, landlord.apartment_id, landlord.end_of_grace_period
    )

    ,Backoffice_2 AS (
        SELECT 
            Backoffice_3.apartment_id, Backoffice_3.city, Backoffice_3.amount, Backoffice_3.month, Backoffice_3.management_accounts,
            CASE 
                WHEN Backoffice_3.month = trunc(end_of_grace_period, 'month') 
                    THEN (COALESCE(CAST(additional_costs_amount AS DECIMAL),0) / EXTRACT(DAY FROM trunc(month, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY)) * (EXTRACT(DAY FROM trunc(month, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY) - EXTRACT(DAY FROM end_of_grace_period))
                WHEN Backoffice_3.amount = 0 THEN 0
                ELSE COALESCE(CAST(additional_costs_amount AS DECIMAL),0)
            END AS additional_costs_amount

        FROM Backoffice_3
            JOIN Additional_costs_2 ON Backoffice_3.apartment_id = Additional_costs_2.apartment_id
    )

    ,Backoffice AS (
        SELECT 
        (SUM(Backoffice_2.amount) + SUM(Backoffice_2.additional_costs_amount)) amount, 
        Backoffice_2.management_accounts, Backoffice_2.city, Backoffice_2.month, apartments.codename 
        FROM Backoffice_2 
        LEFT JOIN silver.backoffice.apartments ON Backoffice_2.apartment_id = silver.backoffice.apartments.id
        GROUP BY Backoffice_2.management_accounts, Backoffice_2.city, Backoffice_2.month, codename
        ORDER BY Backoffice_2.month DESC, Backoffice_2.city ASC
    )

    ,X AS (
    SELECT amount, month, management_accounts, city, codename FROM COGS
    UNION ALL
    SELECT amount, month, management_accounts, city, codename FROM Backoffice
    )

    ,Past AS (
    SELECT * 
    FROM X 
    WHERE month >= trunc(current_date, 'month') - INTERVAL '24 MONTH' AND month <= trunc(current_date, 'month') + INTERVAL '12 MONTH'
    ORDER BY month DESC, management_accounts DESC
    )

    ,Fcst_fc_utilities AS (
        SELECT SUM(amount)*-1 amount, management_accounts, month, city, codename FROM silver.finance.cogs_forecast_utilities_fc as f WHERE true GROUP BY management_accounts, month, city, codename
    )

    ,Fcst_insuarnce AS (
    SELECT amount::DECIMAL * 0.0045 AS amount, month, codename, 'COGS - Rent (€) - Insurance' management_accounts, city FROM Past WHERE management_accounts = 'COGS - Rent (€)'
    )

    ,Series AS (
        SELECT
            distinct management_accounts, city, codename,
            explode(
                sequence(
                    trunc(current_date, 'month') - INTERVAL '24 MONTH',
                    trunc(current_date, 'month') + INTERVAL '12 MONTH',
                    INTERVAL 1 MONTH
                )
            ) month
        
        FROM Past
    )

    SELECT 
        s.management_accounts, s.month, s.city, s.codename,
        COALESCE(SUM(p.amount), SUM(fu.amount), SUM(fi.amount), NULL) amount

    FROM Series s 
        LEFT JOIN Past p ON s.management_accounts = p.management_accounts AND s.month = p.month AND s.city = p.city AND s.codename = p.codename
        LEFT JOIN Fcst_fc_utilities fu ON s.management_accounts = fu.management_accounts AND s.month = fu.month AND s.city = fu.city AND s.codename = fu.codename
        LEFT JOIN Fcst_insuarnce fi ON s.management_accounts = fi.management_accounts AND s.month = fi.month AND s.city = fi.city AND s.codename = fi.codename

    GROUP BY s.management_accounts, s.month, s.codename, s.city
    ORDER BY s.month, s.management_accounts ASC
"""

spark.sql(cogs_per_month_and_city_per_codename).write.mode("overwrite").saveAsTable("silver.finance.cogs_per_month_and_city_per_codename")


### 3.2.2. Deltas 7 Days Ago
###### (COGS Forecast Table)

In [0]:
cogs_per_month_and_city_per_codename_7_days_deltas = """
WITH COGS AS (
        SELECT sum(amount)*-1 amount, to_date(month, 'd/M/yyyy') month, `Management Accounts` AS management_accounts, city, Apartment2 codename 
        FROM silver.finance.gm_by_apt_oct_24 
        WHERE `Line Type` = 'COGS' GROUP BY to_date(month, 'd/M/yyyy'), `Management Accounts`, city, Apartment2
    )

    ,Additional_costs_2 AS (
        SELECT
            ams_contract_terms.apartment_id, city, JSON_TUPLE(RENT_INFO, 'additional_costs_amount') AS additional_costs_amount
        
        FROM silver.backoffice.ams_contract_terms TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) ams_contract_terms
            LEFT JOIN silver.Backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON apartments.id = ams_contract_terms.apartment_id
    )

    ,Backoffice_3 AS (
        SELECT 
        SUM(landlord_daily_rent)::decimal as amount, 
        trunc(day, 'month') as month, 'COGS - Rent (€)' AS management_accounts, city, 
        landlord.apartment_id, landlord.end_of_grace_period
        FROM silver.revenue.landlord TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) landlord 
        LEFT JOIN silver.Backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON apartments.id = landlord.apartment_id
        WHERE trunc(day, 'month') >= trunc(current_date, 'month') - INTERVAL '1 MONTH'
        GROUP BY trunc(day, 'month'), city, landlord.apartment_id, landlord.end_of_grace_period
    )

    ,Backoffice_2 AS (
        SELECT 
            Backoffice_3.apartment_id, Backoffice_3.city, Backoffice_3.amount, Backoffice_3.month, Backoffice_3.management_accounts,
            CASE 
                WHEN Backoffice_3.month = trunc(end_of_grace_period, 'month') 
                    THEN (COALESCE(CAST(additional_costs_amount AS DECIMAL),0) / EXTRACT(DAY FROM trunc(month, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY)) * (EXTRACT(DAY FROM trunc(month, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY) - EXTRACT(DAY FROM end_of_grace_period))
                WHEN Backoffice_3.amount = 0 THEN 0
                ELSE COALESCE(CAST(additional_costs_amount AS DECIMAL),0)
            END AS additional_costs_amount

        FROM Backoffice_3
            JOIN Additional_costs_2 ON Backoffice_3.apartment_id = Additional_costs_2.apartment_id
    )

    ,Backoffice AS (
        SELECT 
        (SUM(Backoffice_2.amount) + SUM(Backoffice_2.additional_costs_amount)) amount, 
        Backoffice_2.management_accounts, Backoffice_2.city, Backoffice_2.month, apartments.codename 
        FROM Backoffice_2 
        LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(CURRENT_DATE() - INTERVAL 7 DAYS AS TIMESTAMP) apartments ON Backoffice_2.apartment_id = apartments.id
        GROUP BY Backoffice_2.management_accounts, Backoffice_2.city, Backoffice_2.month, codename
        ORDER BY Backoffice_2.month DESC, Backoffice_2.city ASC
    )

    ,X AS (
    SELECT amount, month, management_accounts, city, codename FROM COGS
    UNION ALL
    SELECT amount, month, management_accounts, city, codename FROM Backoffice
    )

    ,Past AS (
    SELECT * 
    FROM X 
    WHERE month >= trunc(current_date, 'month') - INTERVAL '24 MONTH' AND month <= trunc(current_date, 'month') + INTERVAL '12 MONTH'
    ORDER BY month DESC, management_accounts DESC
    )

    ,Fcst_fc_utilities AS (
        SELECT SUM(amount)*-1 amount, management_accounts, month, city, codename FROM silver.finance.cogs_forecast_utilities_fc_7_days_deltas as f WHERE true GROUP BY management_accounts, month, city, codename
    )

    ,Fcst_insuarnce AS (
    SELECT amount::DECIMAL * 0.0045 AS amount, month, codename, 'COGS - Rent (€) - Insurance' management_accounts, city FROM Past WHERE management_accounts = 'COGS - Rent (€)'
    )

    ,Series AS (
        SELECT
            distinct management_accounts, city, codename,
            explode(
                sequence(
                    trunc(current_date, 'month') - INTERVAL '24 MONTH',
                    trunc(current_date, 'month') + INTERVAL '12 MONTH',
                    INTERVAL 1 MONTH
                )
            ) month
        
        FROM Past
    )

    SELECT
        s.management_accounts, s.month, s.city, s.codename,
        COALESCE(SUM(p.amount), SUM(fu.amount), SUM(fi.amount), NULL) amount

    FROM Series s 
        LEFT JOIN Past p ON s.management_accounts = p.management_accounts AND s.month = p.month AND s.city = p.city AND s.codename = p.codename
        LEFT JOIN Fcst_fc_utilities fu ON s.management_accounts = fu.management_accounts AND s.month = fu.month AND s.city = fu.city AND s.codename = fu.codename
        LEFT JOIN Fcst_insuarnce fi ON s.management_accounts = fi.management_accounts AND s.month = fi.month AND s.city = fi.city AND s.codename = fi.codename

    GROUP BY s.management_accounts, s.month, s.codename, s.city
    ORDER BY s.month, s.management_accounts ASC
"""

spark.sql(cogs_per_month_and_city_per_codename_7_days_deltas).write.mode("overwrite").saveAsTable("silver.finance.cogs_per_month_and_city_per_codename_7_days_deltas")


### 3.2.3. Deltas Since Begining of Month
###### (COGS Forecast Table)

In [0]:
cogs_per_month_and_city_per_codename_begining_month_deltas = """
WITH COGS AS (
        SELECT sum(amount)*-1 amount, to_date(month, 'd/M/yyyy') month, `Management Accounts` AS management_accounts, city, Apartment2 codename 
        FROM silver.finance.gm_by_apt_oct_24 
        WHERE `Line Type` = 'COGS' GROUP BY to_date(month, 'd/M/yyyy'), `Management Accounts`, city, Apartment2
    )

    ,Additional_costs_2 AS (
        SELECT
            ams_contract_terms.apartment_id, city, JSON_TUPLE(RENT_INFO, 'additional_costs_amount') AS additional_costs_amount
        
        FROM silver.backoffice.ams_contract_terms TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) ams_contract_terms
            LEFT JOIN silver.Backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON apartments.id = ams_contract_terms.apartment_id
    )

    ,Backoffice_3 AS (
        SELECT 
        SUM(landlord_daily_rent)::decimal as amount, 
        trunc(day, 'month') as month, 'COGS - Rent (€)' AS management_accounts, city, 
        landlord.apartment_id, landlord.end_of_grace_period
        FROM silver.revenue.landlord TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) landlord 
        LEFT JOIN silver.Backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON apartments.id = landlord.apartment_id
        WHERE trunc(day, 'month') >= trunc(current_date, 'month') - INTERVAL '1 MONTH'
        GROUP BY trunc(day, 'month'), city, landlord.apartment_id, landlord.end_of_grace_period
    )

    ,Backoffice_2 AS (
        SELECT 
            Backoffice_3.apartment_id, Backoffice_3.city, Backoffice_3.amount, Backoffice_3.month, Backoffice_3.management_accounts,
            CASE 
                WHEN Backoffice_3.month = trunc(end_of_grace_period, 'month') 
                    THEN (COALESCE(CAST(additional_costs_amount AS DECIMAL),0) / EXTRACT(DAY FROM trunc(month, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY)) * (EXTRACT(DAY FROM trunc(month, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY) - EXTRACT(DAY FROM end_of_grace_period))
                WHEN Backoffice_3.amount = 0 THEN 0
                ELSE COALESCE(CAST(additional_costs_amount AS DECIMAL),0)
            END AS additional_costs_amount

        FROM Backoffice_3
            JOIN Additional_costs_2 ON Backoffice_3.apartment_id = Additional_costs_2.apartment_id
    )

    ,Backoffice AS (
        SELECT 
        (SUM(Backoffice_2.amount) + SUM(Backoffice_2.additional_costs_amount)) amount, 
        Backoffice_2.management_accounts, Backoffice_2.city, Backoffice_2.month, apartments.codename 
        FROM Backoffice_2 
        LEFT JOIN silver.backoffice.apartments TIMESTAMP AS OF CAST(trunc(CURRENT_DATE(),'month') AS TIMESTAMP) apartments ON Backoffice_2.apartment_id = apartments.id
        GROUP BY Backoffice_2.management_accounts, Backoffice_2.city, Backoffice_2.month, codename
        ORDER BY Backoffice_2.month DESC, Backoffice_2.city ASC
    )

    ,X AS (
    SELECT amount, month, management_accounts, city, codename FROM COGS
    UNION ALL
    SELECT amount, month, management_accounts, city, codename FROM Backoffice
    )

    ,Past AS (
    SELECT * 
    FROM X 
    WHERE month >= trunc(current_date, 'month') - INTERVAL '24 MONTH' AND month <= trunc(current_date, 'month') + INTERVAL '12 MONTH'
    ORDER BY month DESC, management_accounts DESC
    )

    ,Fcst_fc_utilities AS (
        SELECT SUM(amount)*-1 amount, management_accounts, month, city, codename FROM silver.finance.cogs_forecast_utilities_begining_month_deltas as f WHERE true GROUP BY management_accounts, month, city, codename
    )

    ,Fcst_insuarnce AS (
    SELECT amount::DECIMAL * 0.0045 AS amount, month, codename, 'COGS - Rent (€) - Insurance' management_accounts, city FROM Past WHERE management_accounts = 'COGS - Rent (€)'
    )

    ,Series AS (
        SELECT
            distinct management_accounts, city, codename,
            explode(
                sequence(
                    trunc(current_date, 'month') - INTERVAL '24 MONTH',
                    trunc(current_date, 'month') + INTERVAL '12 MONTH',
                    INTERVAL 1 MONTH
                )
            ) month
        
        FROM Past
    )

    SELECT 
        s.management_accounts, s.month, s.city, s.codename,
        COALESCE(SUM(p.amount), SUM(fu.amount), SUM(fi.amount), NULL) amount

    FROM Series s 
        LEFT JOIN Past p ON s.management_accounts = p.management_accounts AND s.month = p.month AND s.city = p.city AND s.codename = p.codename
        LEFT JOIN Fcst_fc_utilities fu ON s.management_accounts = fu.management_accounts AND s.month = fu.month AND s.city = fu.city AND s.codename = fu.codename
        LEFT JOIN Fcst_insuarnce fi ON s.management_accounts = fi.management_accounts AND s.month = fi.month AND s.city = fi.city AND s.codename = fi.codename

    GROUP BY s.management_accounts, s.month, s.codename, s.city
    ORDER BY s.month, s.management_accounts ASC
"""

spark.sql(cogs_per_month_and_city_per_codename_begining_month_deltas).write.mode("overwrite").saveAsTable("silver.finance.cogs_per_month_and_city_per_codename_begining_month_deltas")


#

# 4. Budget

In [0]:
## USE THE FOLLOWING SPREADSHEET TO UPDATE ACORDING TO WORKING FILE

best_case_budget_pl_volume = """
SELECT account, city, db_month month, volume value FROM silver.finance.raw_working_file_volume_updates
"""

a = spark.sql(best_case_budget_pl_volume)

a.count()

a.repartition(1).write.mode("overwrite").saveAsTable("silver.finance.best_case_budget_pl_volume")
